In [1]:
from config import __database_dir__, contract_list, __labeled_data_dir__
import pickle
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print(__database_dir__)

In [ ]:
species = ['A1', 'A3', 'B2', 'B3']

In [ ]:
def plot_price(file_number, cat):
    with open(__database_dir__ + str(file_number) + '_' + cat, 'rb') as fp:
        timelist, pricelist = pickle.load(fp)
    print('len:', len(timelist))
    plt.plot(pricelist)
    print(timelist[:20])

In [ ]:
number = 0

In [ ]:
plot_price(number, 'A1')

In [ ]:
plot_price(number, 'A3')

In [ ]:
plot_price(number, 'B2')

In [ ]:
plot_price(number, 'B3')

In [ ]:
def get_data(file_number, cat):
    with open(__database_dir__ + str(file_number) + cat, 'rb') as fp:
        data = pickle.load(fp)
    timelist = []
    pricelist = []
    for elem in data:
        timelist.append(elem[0])
        pricelist.append(elem[1])
    return timelist, pricelist

In [ ]:
time, price = get_data(25, 'A1')

In [ ]:
dat = [5, 3, 8, 9, 2]
cal_delta(dat, 0, 1, 5)

In [2]:
from config import train_time, seg_time, predict_st, predict_ed, theta
import numpy as np
import h5py

In [3]:
# 输入: 价格列表, 初始价格下标, 用于标注的区间左端点lo,区间右端点hi,区间为[lo, hi)
# 输出: d_ab(t)
def cal_delta(prices, t0, lo, hi):
    price_st = prices[t0]
    max_delta = 0
    ans = 0
    for i in range(lo, hi):
        delta = prices[i] - price_st
        if abs(delta) > max_delta:
            ans = delta
            max_delta = abs(delta)
    return ans / price_st

In [11]:
# 输入: datelist日期列表(共54天,列表中数字范围[0, 53]), dat_name作为输入的期货种类, lab_name预测的期货种类
# 输出: 将标注后的数据使用h5py导入文件中备用
def label_and_dump(file_num, dat_name, lab_name):
    dataset = []
    label = []
    with open(__database_dir__ + str(file_num) + '_' + dat_name, 'rb') as fp:
        x_time, x_price = pickle.load(fp)
    with open(__database_dir__ + str(file_num) + '_' + lab_name, 'rb') as fp:
        y_time, y_price = pickle.load(fp)
    data_len = int(train_time / 0.5)
    seg_len = int(seg_time / 0.5)
    st_len = int(predict_st / 0.5)
    ed_len = int(predict_ed / 0.5)
    seg_num = min((len(x_time) - data_len) // seg_len, (len(y_time) - data_len - ed_len) // seg_len)
    for i in range(seg_num):
        dataset.append([ x_price[(i+j)*seg_len:seg_len*(i+j+1)] for j in range(data_len // seg_len) ])
        delta = cal_delta(y_price, i*seg_len+data_len, i*seg_len+data_len+st_len, i*seg_len+data_len+ed_len)
        if delta <= -theta:
            label.append(0)
        elif delta >= theta:
            label.append(2)
        else:
            label.append(1)
    with h5py.File(__labeled_data_dir__ + str(file_num) + '_' + dat_name + '_' + lab_name + '.h5', 'w') as fp:
        fp.create_dataset('dataset', data = np.array(dataset))
        fp.create_dataset('label', data = np.array(label))

In [12]:
label_and_dump(52, 'A1', 'A3')

In [6]:
with h5py.File(__labeled_data_dir__ + '52_A1_A3.h5', 'r') as fp:
    data = np.array(fp['dataset'])
    label = np.array(fp['label'])
print(len(data), len(label))
# print(data[:5])
print(label.tolist().count(0))
print(label.tolist().count(1))
print(label.tolist().count(2))

4666 4666
136
4411
119


In [ ]:
with h5py.File("test.h5", "w") as file:
    file.create_dataset('x', data = np.array([1, 2, 3, 4, 5]))
    file.create_dataset('y', data = np.array([1, 0, 1, 0, 1]))

In [ ]:
with h5py.File("test.h5", "r") as file:
    print(np.array(file['x']))
    print(np.array(file['y']))